# Exploratory Data Analysis on F1 using SQL
Queries covered in this notebook:
1. Finding locations that hosted maximum number of race
2. Finding circuits that hosted opening races
3. Number of races hosted by every country
4. All-time constructor points and leaders
5. Number of races organized per season
6. Comparison of all-time top 10 teams (based on points and races won)
7. Number of races won by top 10 drivers
8. Finding the fastest laps on each circuit
9. Number of constructors from each country
10. Finding the fastest qualification times on every circuit for pole position
11. Drivers who have won at least one race

Importing Libraries

In [1]:
import pandas as pd
import sqlite3
import plotly.express as px
import plotly.graph_objects as go
import os 

Establishing Connection Between The SQLITE3 Database 

In [5]:
conn = sqlite3.connect("F1.db")

Reading CSV Files

In [2]:
data_1 = pd.read_csv("Dataset\circuits.csv")
data_2 = pd.read_csv("Dataset\constructor_results.csv")
data_3 = pd.read_csv("Dataset\constructor_standings.csv")
data_4 = pd.read_csv("Dataset\constructors.csv")
data_5 = pd.read_csv("Dataset\driver_standings.csv")
data_6 = pd.read_csv("Dataset\drivers.csv")
data_7 = pd.read_csv("Dataset\lap_times.csv")
data_8 = pd.read_csv("Dataset\pit_stops.csv")
data_9 = pd.read_csv("Dataset\qualifying.csv")
data_10 = pd.read_csv("D:\\Programming\\Projects\\SQL & Tableau\\F1 EDA using SQL & Tableau\\Dataset\\races.csv")
data_11 = pd.read_csv("Dataset\seasons.csv")
data_12 = pd.read_csv("Dataset\status.csv")
data_13 = pd.read_csv("D:\\Programming\\Projects\\SQL & Tableau\\F1 EDA using SQL & Tableau\\Dataset\\results.csv")
data_14 = pd.read_csv("Dataset\sprint_results.csv")

Inserting Data Into SQL Tables 

In [7]:
data_1.to_sql("circuits", conn)
data_2.to_sql("constructor_results", conn)
data_3.to_sql("constructor_standings", conn)
data_4.to_sql("constructors", conn)
data_5.to_sql("driver_standings", conn)
data_6.to_sql("drivers", conn)
data_7.to_sql("lap_times", conn)
data_8.to_sql("pit_stops", conn)
data_9.to_sql("qualifying", conn)
data_10.to_sql("races", conn)
data_11.to_sql("seasons", conn)
data_12.to_sql("status", conn)
data_13.to_sql("results", conn)
data_14.to_sql("sprint_results", conn)

180

# 1. Finding locations that hosted maximum number of races:
Discovering the most popular venues in Formula 1 history by identifying places that have hosted the highest number of races. This question allows us to pinpoint the iconic locations that have been at the heart of numerous thrilling races over the years.

In [8]:
circuit_maximum = pd.read_sql('''
    SELECT c.location AS "City", COUNT(r.circuitid) AS "Races Hosted"
    FROM circuits c, races r 
    WHERE r.circuitid = c.circuitid 
    GROUP BY "City" 
    ORDER BY "Races Hosted" DESC;
''', conn)
circuit_maximum 

fig1 =px.bar(circuit_maximum, x="City", y="Races Hosted", color="Races Hosted", title="Races hosted by cities")
fig1.show()

In [9]:
fig1.write_image("plot1.png")

# 2. Finding The Circuits That Have Hosted The Opening Races:
Uncover the historical significance of Formula 1 by identifying the circuits that hosted the very first races of each season. This question provides insights into the tracks that set the stage for the excitement and competition at the beginning of each racing season.

In [6]:
opening_circuit = pd.read_sql(''' 
        SELECT r.name AS  "Grand Prix Name", c.name AS "Circuit Name", c.location AS "City", c.country AS "Country", count(*) AS "Opening Races Hosted"
        FROM circuits c, races r 
        WHERE r.circuitid = c.circuitid AND r.round = 1
        GROUP BY "City" 
        ORDER BY "Opening Races Hosted" DESC;
''', conn)
opening_circuit

,Grand Prix Name,Circuit Name,City,Country,Opening Races Hosted
0,Australian Grand Prix,Albert Park Grand Prix Circuit,Melbourne,Australia,22
1,Argentine Grand Prix,Autódromo Juan y Oscar Gálvez,Buenos Aires,Argentina,15
2,South African Grand Prix,Kyalami,Midrand,South Africa,8
3,Brazilian Grand Prix,Autódromo Internacional Nelson Piquet,Rio de Janeiro,Brazil,7
4,Bahrain Grand Prix,Bahrain International Circuit,Sakhir,Bahrain,5
5,Monaco Grand Prix,Circuit de Monaco,Monte-Carlo,Monaco,5
6,Brazilian Grand Prix,Autódromo José Carlos Pace,São Paulo,Brazil,3
7,United States Grand Prix,Phoenix street circuit,Phoenix,USA,2
8,Swiss Grand Prix,Circuit Bremgarten,Bern,Switzerland,2
9,Dutch Grand Prix,Circuit Park Zandvoort,Zandvoort,Netherlands,1


In [51]:
fig2 = px.pie(opening_circuit, names='City', values='Opening Races Hosted',
             title='Distribution of Opening Races Hosted by Cities',
             color='Opening Races Hosted')
fig2.show()
fig2.write_image("plot2.png")


# 3. Number Of Races Hosted By Every Country 
Exploring the global landscape of Formula 1 by understanding how many races each country has hosted. This question provides a glimpse into the international appeal of the sport and showcases the diverse locations that have been part of Formula 1 history.

In [7]:
count_races = pd.read_sql('''
        SELECT c.country AS "Country", COUNT(*) AS "Races Hosted" 
        FROM circuits c, races r 
        WHERE r.circuitid = c.circuitid
        GROUP BY "Country"
        ORDER BY "Races Hosted" DESC;
''', conn)
count_races

,Country,Races Hosted
0,Italy,105
1,Germany,79
2,UK,78
3,USA,75
4,Monaco,69
5,Belgium,68
6,France,63
7,Spain,60
8,Canada,52
9,Brazil,50


In [50]:
fig3 = px.treemap(count_races, path=["Country"], values="Races Hosted",
                 title="Countries that have hosted race",
                 labels={"Races Hosted": "Number of Races Hosted"})
fig3.show()
fig3.write_image("plot3.png")

# 4. All Time Constructor Points And Leaders 
Diving into the competitive world of Formula 1 teams by analyzing the cumulative points earned by each team over the years. This question helps us identify the leading 20 teams in the sport, showcasing their sustained success and contribution to Formula 1 history.

In [13]:
constructor_all_time = pd.read_sql('''
        SELECT c.name AS "Constructor", c.nationality AS "Nationality", SUM(points) AS "Total Points"
        FROM constructor_results r, constructors c
        WHERE c.constructorId = r.constructorId
        GROUP BY "Constructor"
        ORDER BY "Total Points" DESC
        LIMIT 20;
''', conn)
constructor_all_time

,Constructor,Nationality,Total Points
0,Ferrari,Italian,9505.0
1,Mercedes,German,7060.5
2,Red Bull,Austrian,6891.0
3,McLaren,British,6191.5
4,Williams,British,3609.0
5,Renault,French,1777.0
6,Force India,Indian,1098.0
7,Team Lotus,British,918.0
8,Benetton,Italian,861.5
9,Lotus F1,British,706.0


In [15]:
fig4 = px.bar(constructor_all_time, x="Constructor", y="Total Points", color="Total Points", title="Top 20 Constructors - All-time Points Scored")
fig4.show()
fig4.write_image("plot4.png")

# 5. Finding Number Of Races Organized Per Season
Examining the evolution of Formula 1 over time by understanding the variation in the number of races organized each season. This question allows us to identify trends and patterns in the scheduling of races throughout the history of the sport.

In [16]:
race_per_season = pd.read_sql('''
        SELECT strftime("%Y", "date") AS "Year", COUNT(*) AS "Races held"
        FROM races
        GROUP BY "Year"
        ORDER BY "Year";
''', conn)
race_per_season

,Year,Races held
0,1950,7
1,1951,8
2,1952,8
3,1953,9
4,1954,9
...,...,...
69,2019,21
70,2020,17
71,2021,22
72,2022,22


In [17]:
fig5 = px.line(race_per_season, x="Year", y="Races held", title="Races Held Every Season")
fig5.show()
fig5.write_image("plot5.png")

# 6. Comparing All-Time Top 10 Teams (Based On Points And Races Won)
Gaining insights into the performance of the top 10 Formula 1 teams by comparing their total points and the number of races they have won. This question provides a comprehensive view of the most successful teams in Formula 1 history.

In [17]:
constructor_top = pd.read_sql('''
        SELECT c.name AS "Constructor", c.nationality AS "Nationality", SUM(points) AS "Total Points"
        FROM constructor_results r, constructors c 
        WHERE c.constructorId = r.constructorId AND points > 0
        GROUP BY "Constructor" 
        ORDER BY "Total Points" DESC
        limit 10;
''', conn)
constructor_top

,Constructor,Nationality,Total Points
0,Ferrari,Italian,9505.0
1,Mercedes,German,7060.5
2,Red Bull,Austrian,6891.0
3,McLaren,British,6191.5
4,Williams,British,3609.0
5,Renault,French,1777.0
6,Force India,Indian,1098.0
7,Team Lotus,British,918.0
8,Benetton,Italian,861.5
9,Lotus F1,British,706.0


In [49]:
fig6 = px.bar(constructor_top, x='Total Points', y='Constructor',
             title='Top 10 Teams Of ALL-Time',
             color='Total Points',
             labels={'Total Points': 'Total Points', 'Constructor': 'Team'},
             orientation='h',
             category_orders={"Constructor": constructor_top["Constructor"].tolist()})
fig6.show()
fig6.write_image("Plot6.png")


# 7. Finding The Number Of Races Won By Top 10 Drivers
Exploring the dominance of the top 10 drivers in Formula 1 by examining the number of races they have won. This question highlights the achievements of the sport's most successful drivers and their impact on Formula 1 history.

In [21]:
drivers_top = pd.read_sql('''
        SELECT ("forename" || " " || "surname") AS "Name", COUNT(r.driverId) AS "Races Won" 
        FROM drivers d, results r
        WHERE r.driverId = d.driverId AND r.position = 1
        GROUP BY "Name"
        ORDER BY "Races Won" DESC
        LIMIT 10;
''', conn)
drivers_top

,Name,Races Won
0,Lewis Hamilton,103
1,Michael Schumacher,91
2,Sebastian Vettel,53
3,Alain Prost,51
4,Max Verstappen,45
5,Ayrton Senna,41
6,Fernando Alonso,32
7,Nigel Mansell,31
8,Jackie Stewart,27
9,Niki Lauda,25


In [47]:
fig7 = px.bar(drivers_top, x='Races Won', y='Name', title='Races Won By Top 10 Drivers',
             color='Races Won', orientation='h',
             labels={'Races Won': 'Number of Races Won', 'Name': 'Driver'})
fig7.show()
fig7.write_image('Plot7.png')

# 8. Finding The Fastest Laps On Each Circuit
Discovering the thrilling speed of Formula 1 by identifying the fastest laps recorded on each circuit. This question showcases the exceptional performance of drivers on specific tracks, emphasizing their skill and speed.

In [29]:
fastest_lap = pd.read_sql('''
        SELECT c.location AS "City", r.year AS "Year of race", MIN(l.time) AS "Lap Time", ("forename" || " " || "surname") AS "Driver"
        FROM circuits c, lap_times l, races r, drivers d 
        WHERE l.raceId = r.raceId AND r.circuitId = c.circuitId AND l.driverId = d.driverId 
        GROUP BY "City"
        ORDER BY "City";
''', conn)
fastest_lap

,City,Year of race,Lap Time,Driver
0,Abu Dhabi,2021,1:26.103,Max Verstappen
1,Al Daayen,2021,1:23.196,Max Verstappen
2,Austin,2019,1:36.169,Charles Leclerc
3,Baku,2019,1:43.009,Charles Leclerc
4,Budapest,2001,19:29.677,Olivier Panis
5,Buenos Aires,1996,15:41.618,Mika Salo
6,Estoril,1996,1:22.873,Jacques Villeneuve
7,Hockenheim,2005,14:06.799,Mark Webber
8,Imola,2006,13:29.130,Yuji Ide
9,Indianapolis,2004,1:10.399,Rubens Barrichello


In [24]:
fig8 = px.bar(fastest_lap, x='City', y= 'Lap Time',hover_data='Driver', title='Fastest Lap Time By Circuit')
fig8.show()
fig8.write_image('plot8.png')

# 9. Number of Constructors (Teams) From Each Country
Understanding the international composition of Formula 1 teams by examining the number of constructors from each country. This question provides insights into the diverse origins of teams and the global nature of Formula 1.

In [33]:
constructor_num = pd.read_sql('''
        SELECT nationality AS "Nationality", COUNT(*) AS "Number of constructors"
        FROM constructors
        GROUP BY "Nationality"
        ORDER BY "Number of constructors" DESC;
''', conn)
constructor_num

,Nationality,Number of constructors
0,British,86
1,American,39
2,Italian,29
3,French,13
4,German,10
5,Swiss,5
6,Japanese,5
7,South African,3
8,Dutch,3
9,Russian,2


In [46]:
fig9= px.treemap(constructor_num, path=['Nationality'], values='Number of constructors',
                 title='Constructors From Different Nationalities',
                 color='Number of constructors')
fig9.show()
fig9.write_image('Plot9.png')

# 10. Finding The Fastest Qualification Time For Pole Position For Every Race 
Exploring the excitement of Formula 1 qualifying sessions by identifying the fastest qualification times on each circuit. This question highlights the drivers who secured the coveted pole position with the quickest laps.

In [27]:
fastest_quali = pd.read_sql('''
        SELECT r.name AS "Grand Prix Name", ("forename" || " " || "surname") AS "Driver", MIN(q1) AS "Fastest Q1 time", MIN(q2) AS "Fastest Q2 time", MIN(Q3) AS "Fastest Q3 time" 
        FROM qualifying q, races r, drivers d 
        WHERE q.raceId = r.raceId AND q.position = 1 AND q.driverId = d.driverId 
        GROUP BY "Grand Prix Name"
        ORDER BY "Grand Prix Name";
''', conn)
fastest_quali

,Grand Prix Name,Driver,Fastest Q1 time,Fastest Q2 time,Fastest Q3 time
0,70th Anniversary Grand Prix,Valtteri Bottas,1:26.738,1:25.785,1:25.154
1,Abu Dhabi Grand Prix,Max Verstappen,1:23.322,1:22.800,1:22.109
2,Argentine Grand Prix,David Coulthard,1:24.473,\N,\N
3,Australian Grand Prix,Max Verstappen,1:16.179,1:17.056,1:16.732
4,Austrian Grand Prix,Valtteri Bottas,1:04.111,1:03.015,1:02.939
5,Azerbaijan Grand Prix,Charles Leclerc,1:41.269,1:41.037,1:40.203
6,Bahrain Grand Prix,Lewis Hamilton,1:28.343,1:27.586,1:27.264
7,Belgian Grand Prix,Lewis Hamilton,1:42.323,1:41.553,1:41.252
8,Brazilian Grand Prix,Lewis Hamilton,1:08.242,1:07.503,1:07.281
9,British Grand Prix,Sebastian Vettel,1:18.233,1:18.119,1:19.509


In [28]:
fig10 = px.bar(fastest_quali, x='Grand Prix Name', y='Fastest Q3 time', title='Fastest Qualification Time for Pole Position in Formula 1')
fig10.update_layout(yaxis_title='Fastest Q3 Time (seconds)') 
fig10.show()
fig10.write_image('plot10.png')

# 11. Finding Driver Who Has Won At Least One Race 
Celebrating the diverse talent in Formula 1 by compiling a list of drivers who have achieved the milestone of winning at least one race. This question showcases the broad spectrum of successful drivers who have left their mark on the sport's rich history.

In [40]:
race_won = pd.read_sql('''
        SELECT ("forename" || " " || "surname") AS "Driver", SUM(case when position = 1 then 1 else 0 end) AS "Races Won" 
        FROM drivers d, results r
        WHERE r.driverId = d.driverId AND position = 1
        GROUP BY "Driver"
        ORDER BY "Races Won" DESC;
''', conn)
race_won

,Driver,Races Won
0,Lewis Hamilton,103
1,Michael Schumacher,91
2,Sebastian Vettel,53
3,Alain Prost,51
4,Max Verstappen,45
...,...,...
108,Esteban Ocon,1
109,Carlos Sainz,1
110,Carlos Pace,1
111,Bob Sweikert,1


In [30]:
fig11=px.bar(race_won, x="Driver", y="Races Won", color="Races Won", title="Drivers Who Have Won The Race")
fig11.show()
fig11.write_image('plot11.png')